In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import os

# Cargar los datasets
# Asegúrate de que la ruta sea correcta, asumiendo que ml-1m está en data/ml-1m/
movies_path = '../data/ml-1m/movies.dat'
ratings_path = '../data/ml-1m/ratings.dat'
users_path = '../data/ml-1m/users.dat'

# Definir nombres de columnas y separadores
unames = ['user_id', 'gender', 'age', 'occupation', 'zip_code']
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
mnames = ['movie_id', 'title', 'genres']

users = pd.read_csv(users_path, sep='::', header=None, names=unames, engine='python')
ratings = pd.read_csv(ratings_path, sep='::', header=None, names=rnames, engine='python')
movies = pd.read_csv(movies_path, sep='::', header=None, names=mnames, engine='python', encoding='latin-1')

# Unir los datasets
df = pd.merge(ratings, users, on='user_id')
df = pd.merge(df, movies, on='movie_id')

print('Shape of merged dataframe:', df.shape)
print('First 5 rows of merged dataframe:')
print(df.head())

In [ ]:
# Limpieza de datos
# Eliminar duplicados (si los hubiera, aunque en MovieLens 1M no suelen haber ratings duplicados exactos)
df.drop_duplicates(inplace=True)
print(f'Shape after dropping duplicates: {df.shape}')

# Verificar IDs inválidos (por ejemplo, IDs que no existen en las tablas originales)
# Esto ya se maneja implícitamente con el merge, pero podemos verificar si hay nulos después del merge
print('
Missing values after merge:')
print(df.isnull().sum())

In [ ]:
# Análisis exploratorio de datos (EDA)

# 1. Distribución de ratings
plt.figure(figsize=(10, 6))
sns.countplot(x='rating', data=df, palette='viridis')
plt.title('Distribución de Ratings')
plt.xlabel('Rating')
plt.ylabel('Número de Ratings')
plt.show()

# 2. Top usuarios por número de ratings
top_users = df['user_id'].value_counts().head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_users.index, y=top_users.values, palette='magma')
plt.title('Top 10 Usuarios por Número de Ratings')
plt.xlabel('ID de Usuario')
plt.ylabel('Número de Ratings')
plt.show()

# 3. Top películas por número de ratings
top_movies = df['title'].value_counts().head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_movies.index, y=top_movies.values, palette='plasma')
plt.title('Top 10 Películas por Número de Ratings')
plt.xlabel('Título de la Película')
plt.ylabel('Número de Ratings')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# 4. Longitud de títulos (para contenido)
df['title_length'] = df['title'].apply(len)
plt.figure(figsize=(10, 6))
sns.histplot(df['title_length'], bins=30, kde=True, color='skyblue')
plt.title('Distribución de la Longitud de los Títulos de Películas')
plt.xlabel('Longitud del Título')
plt.ylabel('Frecuencia')
plt.show()

# 5. Wordcloud de géneros
all_genres = ' '.join(df['genres'].str.replace('|', ' ', regex=False))
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_genres)
plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Nube de Palabras de Géneros de Películas')
plt.show()